In [ ]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

# Muli-task learning
This notebook mostly mirrors the simple_captions notebook but predicts three different outputs instead of just captions.

In [1]:
import json

with open('data/data.json') as infile:  
    data = json.load(infile)

In [2]:
len(data['moves'])

78869

In [4]:
def pieceToWord(notation, verbose=False):
    '''
    Returns a short sentence describing a piece and its position
    '''
    
    pieces = {
        "B": "Bishop",
        "R": "Rook",
        "K": "King",
        "Q": "Queen",
        "N": "Knight"
    }
    
    length = len(notation)
    
    if length == 0:
        return "Pawn"
    
    if length == 1:
        if notation.islower(): # Pawn
            if verbose:
                return "Pawn from file " + notation
            else:
                return "Pawn"
        else: # Higher piece
            return pieces[notation]
    
    piece = pieces[notation[0]]
    
    if not verbose:
        return piece
    
    if length == 2:
        if notation[1].isdigit(): # rank
            return piece + " from rank " + notation[1]
        else: # file
            return piece + " from file " + notation[1]
    
    if length == 3: # rank and file
        return piece + " from square " + notation[1:] 

def moveToSentence(move, verbose=False):
    '''
    Translates a move in the algebraic notation into a caption.
    '''
    
    pieces = {
        "B": "Bishop",
        "R": "Rook",
        "K": "King",
        "Q": "Queen",
        "N": "Knight"
    }
    
    action = " to " # Default action is move
    modifier = "" # Default no modifier
    
    if "+" in move: # Check
        if verbose:
            modifier = " with check"
        move = move[:-1]
    
    if "#" in move: # Checkmate
        if verbose:
            modifier = " with checkmate"
        move = move[:-1]
        
    if "=" in move: # Pawn promotion
        if verbose:
            modifier = " with promotion to " + pieces[move[-1:]].lower() + modifier
        move = move[:-2]
        
    if "e.p." in move: # En passant
        move = move.replace('e.p.', '')
        if verbose:
            return  " en passant" + modifier
    
    # Special cases
    if "O-O" == move: # Kingside caste
        return "Kingside castle" + modifier
    
    if move == "O-O-O": # Queenside castle
        return "Queenside castle" + modifier    
        
    # extract final position
    position = move[-2:]
    move = move[:-2]
        
    if "x" in move: # Capture
        action = " takes on "
        move = move[:-1]
        
    piece = pieceToWord(move, verbose)
    
    return piece + action + position + modifier + " <End>"

moveToSentence("dxe8=Q+", True)

'Pawn from file d takes on e8 with promotion to queen with check <End>'

In [5]:
# Helper methods to convert fens to the matrix representation
import numpy as np

def indexToArray(i, len_ = 13):
    '''
    Converts an index into a one-hot-encoded vector.
    i - the index of the 1.
    len_ - (optional) the len of the one-hot-vector. Default is 12.
    returns a vector of length len_
    '''
    
    array = [0] * len_
    
    if(i >= 0 and i < len(array)):
        array[i] = 1
        
    return array

def fenToMatrix(fen):
    '''
    Converts a fen string to a 8x8x16 matrix.
    fen - a string in the fen notation
    returns a 8x8x12 matrix
    '''
    
    # 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
    pieces = {
        '': indexToArray(0),
        'r': indexToArray(1),
        'n': indexToArray(2),
        'b': indexToArray(3),
        'q': indexToArray(4),
        'k': indexToArray(5),
        'p': indexToArray(6),
        'P': indexToArray(7),
        'R': indexToArray(8),
        'N': indexToArray(9),
        'B': indexToArray(10),
        'Q': indexToArray(11),
        'K': indexToArray(12),
    }
    
    matrix = []
    row = []
    
    for c in fen:
        try:
            cInt = int(c)
            
            for i in range(cInt):
                row.append(pieces[''])
        except: # c can not be cast as integer
            if c == '/':
                matrix.append(row)
                row = []
            else:
                row.append(pieces[c]) 
    matrix.append(row)
                
    return np.array(matrix)

def matrixToFen(matrix):
    '''
    Converts 8x8x13 matrix to a fen string.
    matrix - a 8x8x13 matrix
    returns a fen string
    '''
    
    # 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
    pieces = [
        '',
        'r',
        'n',
        'b',
        'q',
        'k',
        'p',
        'P',
        'R',
        'N',
        'B',
        'Q',
        'K',
    ]
        
    matrix = (matrix > 0).astype(int) * (matrix == matrix.max(axis=2)[:,:,None]).astype(int)
        
    fen = ""
     
    for i in range(8):
        row = matrix[i]
        empty = 0
        
        for j in range(8):
            square = row[j]
            piece_found = False
            
            for k in range(len(pieces)):
                piece = square[k]
                
                if piece == 1:
                    if pieces[k] == '':
                        empty += 1
                    else:
                        if empty > 0:
                            fen += str(empty)
                            empty = 0
                        fen += pieces[k]               
                
        if empty > 0:
            fen += str(empty)  
            
        if i < 7:
            fen += "/"  
            
    return fen

def similarityScore(y_true, y_pred):
    '''
    Calculates the similarity between to chess position
    The result is the number of squares that have the same piece/are empty accross the two boards
    y_true - an 8x8x12 matrix
    y_pred - and 8x8x12 matrix
    returns a similarity between 0 and 1
    '''
    
    score = 0
    
    y_true = (y_true > 0).astype(int) * (y_true == y_true.max(axis=2)[:,:,None]).astype(int)
    y_pred = (y_pred > 0).astype(int) * (y_pred == y_pred.max(axis=2)[:,:,None]).astype(int)
    
    diff = (y_true - y_pred) ** 2
    
    wrong = np.sum(diff) 
    
    return 1 - (wrong / 32)  # Two boards can at most have 32 different fields (or they can have 16 fields where different units are places which also results in a wrong-count of 32) 
    
print(data['fens'][0][0])
print(matrixToFen(fenToMatrix(data['fens'][0][0])))

rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR
rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR


In [7]:
verbose = True

pieces = ["Pawn", "Rook", "Knight", "Bishop", "Queen", "King"]
files = list('abcdefgh')
ranks = list('12345678')
squares = [f+r for r in ranks for f in files]

# Basic words in sentence
vocab = {
    "<Pad>": 0,
    "<End>": 1,
    "Kingside": 2,
    "castle": 3,
    "Queenside": 4,
    "takes": 5,
    "on": 6,   
    "to": 7,
}

if verbose:
    vocab["en"] = 8
    vocab["passant"] = 9
    vocab["with"] = 10
    vocab["check"] = 11
    vocab["checkmate"] = 12
    vocab["promotion"] = 13
    vocab["from"] = 14
    vocab["file"] = 15
    vocab["rank"] = 16
    vocab["square"] = 17
    
    # Files
    for file in files:
        vocab[file] = len(vocab)

    # Ranks
    for rank in ranks:
        vocab[rank] = len(vocab)
        
# Pieces
for piece in pieces:
    vocab[piece] = len(vocab)
    vocab[piece.lower()] = len(vocab)
    
# Squares
for square in squares:
    vocab[square] = len(vocab)
    
len(vocab)

110

In [8]:
from tqdm import tqdm_notebook as tqdm

combined = []

for i in tqdm(range(len(data['fens']))):
    game = data['moves'][i]
    fens = data['fens'][i]
    
    start = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"
    
    for j in range(len(game)):
        if j == 0: # First move
            fen = [start, fens[j]]
        else:
            fen = [fens[j-1], fens[j]]
            
        sentence = moveToSentence(game[j]).split(' ')
        
        for i in range(len(sentence)):
            combined.append([i, game[j], fen])
                
print("Combined", len(combined))


Combined 24083414


In [9]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, combined, vocab, verbose):
        'Initialization'
        self.batch_size = 64
        self.combined = combined
        self.vocab = vocab
        self.verbose = verbose

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.combined) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        combined = self.combined[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(combined)

        return X, y

    def __data_generation(self, combined):
        'Generates data containing batch_size samples' 
        from_matrices = []
        to_matrices = []
        sentences = []
        targets = []
             
        for i, move, fens in combined:
            from_matrix = fenToMatrix(fens[0])
            to_matrix = fenToMatrix(fens[1])
            
            from_matrices.append(from_matrix)
            to_matrices.append(to_matrix)
            
            sentence = moveToSentence(move, self.verbose).split(' ')
            sentence = [self.vocab[word] for word in sentence]
            
            target = np.zeros(len(vocab))
            target[sentence[i]] = 1
            sentence = sentence[:i]
            sentence = sentence + [0] * (15 - len(sentence)) # Add padding

            sentences.append(sentence)
            targets.append(target)
            
        from_matrices = np.array(from_matrices)
        to_matrices = np.array(to_matrices)
        sentences = np.array(sentences)
        targets = np.array(targets)
            
        return [from_matrices, to_matrices, sentences], [to_matrices, targets]
    
np.random.shuffle(combined)
training_generator = DataGenerator(combined[:23000000], vocab, verbose)
validation_generator = DataGenerator(combined[23000000:], vocab, verbose)

Using TensorFlow backend.


In [10]:
training_generator[0][1][0].shape

(64, 8, 8, 13)

In [11]:
from keras.models import load_model

model = load_model('autoencoder1.h5')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 13)          0         
_________________________________________________________________
model_2 (Model)              (None, 256)               1509120   
_________________________________________________________________
model_1 (Model)              (None, 8, 8, 13)          1509696   
Total params: 3,018,816
Trainable params: 3,018,816
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
from keras.models import Model
from keras.layers import Dense, Activation, Input, Flatten, Reshape, GRU, Embedding, concatenate
from keras import metrics

def Encoder():
    from_matrix = Input(shape=(8, 8, 13))
    to_matrix = Input(shape=(8, 8, 13))
    sentence = Input(shape=(10,))
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder1'
    encoder.layers.pop(0)
    x1 = encoder(from_matrix)
    x1 = Model(inputs=from_matrix, outputs=x1)
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder2'
    encoder.layers.pop(0)
    x2 = encoder(to_matrix)
    x2 = Model(inputs=to_matrix, outputs=x2)

    y = Embedding(len(vocab), 50, mask_zero=True)(sentence)
    y = GRU(124, activation='relu')(y)
    y = Model(inputs=sentence, outputs=y)
    
    z = concatenate([x1.output, x2.output, y.output])
    
    return Model(inputs=[x1.input, x2.input, y.input], outputs=z)

def Decoder():
    X = Input(shape=(636,))
    
    # Board state
    d11 = Dense(1024, activation='relu')(X)
    d12 = Dense(8*8*13, activation='linear')(d11)
    r1 = Reshape((8, 8, 13), name="board")(d12)
    
    # Caption
    d21 = Dense(256, activation='relu')(X)
    d22 = Dense(len(vocab), activation='softmax', name="caption")(d21)
    
    return Model(X, [r1, d22])

losses = ["categorical_crossentropy", "categorical_crossentropy"]
metrics = {
    "board": "categorical_accuracy",
    "caption": "categorical_accuracy",
}

# define input to the model:
X = [Input(shape=(8, 8, 13)), Input(shape=(8, 8, 13)), Input(shape=(None,))]

# make the model:
multi_task_model = Model(X, Decoder()(Encoder()(X)))

# compile the model:
multi_task_model.compile(optimizer='adam', loss=losses, metrics=metrics)

In [13]:
multi_task_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 636)          3088840     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [14]:
from keras_tqdm import TQDMNotebookCallback

# Train model on dataset
multi_task_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=1, verbose=0, callbacks=[TQDMNotebookCallback()])

Epoch 1/1
    50/359375 [..............................] - ETA: 17:01:20 - loss: 14.5171 - model_1_loss: 3.4003

KeyboardInterrupt: 